In [ ]:
# --- Bagian 0: Persiapan Awal ---

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import numpy as np
import json
from typing import List

print("Library berhasil diimpor.")

In [ ]:
# --- Bagian 1: Muat Data ---
# Muat data kasus yang sudah diproses dari Tahap 2
try:
    df_cases = pd.read_csv('data/processed/cases.csv')
    print("Data kasus berhasil dimuat.")
    print(f"Jumlah dokumen: {len(df_cases)}")
    print("Contoh 5 baris pertama:")
    print(df_cases.head())
    print(f"Kolom yang tersedia: {df_cases.columns.tolist()}")

    # Pastikan kolom 'text full' ada dan tidak kosong
    if 'text full' not in df_cases.columns:
        raise ValueError("Kolom 'text full' tidak ditemukan di cases.csv. Pastikan nama kolomnya benar.")
    if df_cases['text full'].isnull().any():
        print("Peringatan: Ada nilai kosong di kolom 'text full'. Akan diisi dengan string kosong.")
        df_cases['text full'] = df_cases['text full'].fillna('')

except FileNotFoundError:
    print("Error: File 'data/processed/cases.csv' tidak ditemukan.")
    print("Pastikan Anda telah menjalankan Tahap 2 (Case Representation) dan menyimpan outputnya.")
    raise 
except ValueError as e:
    print(f"Error: {e}")
    raise 

corpus = df_cases['text full'].tolist()
case_ids = df_cases['case id'].tolist()

In [ ]:
## i. Representasi Vektor (TF-IDF)

In [ ]:
# --- Bagian 2: Representasi Vektor (TF-IDF) ---

print("\n--- Bagian 2: Representasi Vektor (TF-IDF) ---")

tfidf_vectorizer = TfidfVectorizer(max_features=5000, min_df=5, max_df=0.8)
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

print(f"Bentuk TF-IDF matrix: {tfidf_matrix.shape} (Jumlah dokumen x Jumlah fitur)")
print(f"Contoh sebagian fitur (kata): {tfidf_vectorizer.get_feature_names_out()[:20]}...")
print("Representasi vektor TF-IDF berhasil dibuat.")

In [ ]:
## ii. Splitting Data

In [ ]:
## iii. Model Retrieval (Cosine Similarity pada TF-IDF)

In [ ]:
# --- Bagian 4: Model Retrieval (Cosine Similarity pada TF-IDF) ---

print("\n--- Bagian 4: Model Retrieval (Cosine Similarity pada TF-IDF) ---")

def calculate_similarities(query_vector, case_vectors):
    similarities = cosine_similarity(query_vector.reshape(1, -1), case_vectors)
    return similarities.flatten()

print("Fungsi 'calculate_similarities' siap digunakan.")

In [ ]:
## iv. Fungsi Retrieval

In [ ]:
# --- Bagian 5: Implementasi Fungsi 'retrieve()' ---

print("\n--- Bagian 5: Implementasi Fungsi 'retrieve()' ---")

def retrieve(query: str, k: int = 5, vectorizer: TfidfVectorizer = tfidf_vectorizer,
             case_vectors_db: np.ndarray = tfidf_matrix,
             case_id_list: List[str] = case_ids) -> List[str]:
    """
    Mengambil top-k kasus yang paling mirip dengan query dari database kasus.

    Args:
        query (str): Teks query (kasus baru).
        k (int): Jumlah kasus paling mirip yang ingin dikembalikan.
        vectorizer (TfidfVectorizer): Objek TfidfVectorizer yang sudah dilatih.
        case_vectors_db (np.ndarray): Matriks vektor kasus (TF-IDF matrix) dari seluruh case base.
        case_id_list (List[str]): List ID kasus yang sesuai dengan baris di case_vectors_db.

    Returns:
        List[str]: List berisi ID dari top-k kasus yang paling mirip.
    """
    # 1) Pre-process query (lakukan preprocessing yang sama seperti pada data kasus)
    processed_query = query.lower()

    # 2) Hitung vektor query
    query_vector = vectorizer.transform([processed_query])

    # 3) Hitung cosine-similarity dengan semua case vectors
    similarities = cosine_similarity(query_vector, case_vectors_db).flatten()

    # 4) Kembalikan top-k case_id
    top_k_indices = np.argsort(similarities)[::-1][:k]
    top_k_case_ids = [case_id_list[i] for i in top_k_indices]

    return top_k_case_ids

print("Fungsi 'retrieve()' berhasil didefinisikan.")

In [ ]:
## v. Pengujian Awal

In [ ]:
# --- Bagian 6: Pengujian Awal Fungsi 'retrieve()' ---

print("\n--- Bagian 6: Pengujian Awal Fungsi 'retrieve()' ---")

test_queries = {
    "query_1": {
        "text": "Seorang terdakwa dihukum 3 tahun penjara dan denda 800 juta karena kepemilikan sabu seberat 0,6898 gram.",
        "ground_truth_case_ids": ["4435kpid.sus2025"]
    },
    "query_2": {
        "text": "Kasus pidana narkotika dengan pasal 112 ayat 1 UU nomor 35 tahun 2009 dan hukuman 1 tahun penjara.",
        "ground_truth_case_ids": ["3252kpid.sus2025", "3016kpid.sus2025"]
    },
    "query_3": {
        "text": "Terdakwa terbukti membawa narkotika jenis ganja dan dijerat pasal 111 ayat 1 UU Narkotika.",
        "ground_truth_case_ids": ["3126kpid.sus2025", "4204kpid.sus2025"]
    },
    "query_4": {
        "text": "Pelaku penyalahgunaan sabu dengan hukuman 7 tahun penjara dan denda 1 miliar rupiah.",
        "ground_truth_case_ids": ["2179kpid.sus2025", "2989kpid.sus2025"]
    },
    "query_5": {
        "text": "Putusan terkait pasal 132 ayat 1 undang-undang nomor 35 tahun 2009 tentang narkotika dengan denda besar.",
        "ground_truth_case_ids": ["2355kpid.sus2025", "3097kpid.sus2025"]
    }
}

try:
    with open('data/eval/queries.json', 'w') as f:
        json.dump(test_queries, f, indent=4)
    print("\nFile 'data/eval/queries.json' berhasil dibuat.")
except Exception as e:
    print(f"Error saat menyimpan queries.json: {e}")

print("\n--- Hasil Pengujian Fungsi 'retrieve()' ---")
for query_name, q_data in test_queries.items():
    query_text = q_data['text']
    retrieved_ids = retrieve(query_text, k=5,
                             vectorizer=tfidf_vectorizer,
                             case_vectors_db=tfidf_matrix,
                             case_id_list=case_ids)

    print(f"\nQuery ({query_name}): \"{query_text}\"")
    print(f"Ground Truth Case IDs: {q_data['ground_truth_case_ids']}")
    print(f"Retrieved Top 5 Case IDs: {retrieved_ids}")

    hits = [id for id in retrieved_ids if id in q_data['ground_truth_case_ids']]
    print(f"Hits (ground truth ditemukan): {hits}")
    print(f"Proporsi hit: {len(hits) / len(q_data['ground_truth_case_ids']):.2f}")

In [ ]:
## vi. Output Tahap Retrieval

In [ ]:
# --- Bagian 7: Output Tahap Retrieval ---

print("\n--- Bagian 7: Output Tahap Retrieval ---")
print("Pastikan Anda menyimpan notebook ini sebagai 'notebooks/03_case_retrieval.ipynb'.")
print("Fungsi 'retrieve()' telah didefinisikan dan diuji.")
print("File 'data/eval/queries.json' telah dibuat.")